# Setup

In [11]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [12]:
import requests
import os
import re
from openai import OpenAI
from dotenv import load_dotenv
import phoenix as px
from phoenix.otel import register
from openinference.instrumentation.openai import OpenAIInstrumentor


In [13]:
# Launch Phoenix
px.launch_app()

# defaults to endpoint="http://localhost:4317"
tracer_provider = register(
  project_name="fourth_wall_app", # Default is 'default'
  endpoint="http://localhost:4317",  # Sends traces using gRPC
) 

OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: fourth_wall_app
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



Unknown project: UHJvamVjdDoy

GraphQL request:4:3
3 | ) {
4 |   node(id: $id) {
  |   ^
5 |     __typename
Traceback (most recent call last):
  File "/home/pineatwork/fourthwall/.venv/lib/python3.11/site-packages/graphql/execution/execute.py", line 530, in await_result
    return_type, field_nodes, info, path, await result
                                          ^^^^^^^^^^^^
  File "/home/pineatwork/fourthwall/.venv/lib/python3.11/site-packages/strawberry/schema/schema_converter.py", line 750, in _async_resolver
    return await await_maybe(
           ^^^^^^^^^^^^^^^^^^
  File "/home/pineatwork/fourthwall/.venv/lib/python3.11/site-packages/strawberry/utils/await_maybe.py", line 12, in await_maybe
    return await value
           ^^^^^^^^^^^
  File "/home/pineatwork/fourthwall/.venv/lib/python3.11/site-packages/phoenix/server/api/queries.py", line 460, in node
    raise NotFound(f"Unknown project: {id}")
phoenix.server.api.exceptions.NotFound: Unknown project: UHJvamVjdDoy
Unknown 

In [14]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(
    api_key=api_key
)

# Gather Story Summary

## Wikipedia

In [15]:
def get_wikipedia_summary(title):
    """
    Fetches the first extract of a Wikipedia article using the public MediaWiki API.
    Returns a text summary or an empty string if not found.
    """
    base_url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "prop": "extracts",
        "explaintext": True,
        "format": "json",
        "titles": title
    }
    try:
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        pages = data.get("query", {}).get("pages", {})
        for page_id, page_content in pages.items():
            if "extract" in page_content:
                # This is a raw textual extract from Wikipedia
                return page_content["extract"]
    except requests.RequestException as e:
        print(f"[Wikipedia] Error fetching summary: {e}")

    return ""

In [16]:
# testing with my fav solarpunk novel
book = "The Windup Girl"
summary = get_wikipedia_summary(book)
print(summary)

The Windup Girl is a biopunk science fiction novel by American writer Paolo Bacigalupi. It was his debut novel and was published by Night Shade Books on September 1, 2009. The novel is set in a future Thailand and covers a number of contemporary issues such as global warming and biotechnology.
The Windup Girl was named as the ninth best fiction book of 2009 by Time magazine. It won the 2010 Nebula Award and the 2010 Hugo Award (tied with The City & the City by China Miéville), both for best novel. The book also won the 2010 Campbell Memorial Award, the 2010 Compton Crook Award and the 2010 Locus Award for best first novel.


== Setting ==
The Windup Girl is set in 23rd-century Thailand. Global warming has raised the levels of world's oceans, carbon fuel sources have become depleted, and manually wound springs are used as energy storage devices. Biotechnology is dominant and megacorporations (called calorie companies) like AgriGen, PurCal and RedStar control food production through "gen

In [17]:
# from llama_index import GPTSimpleVectorIndex, SimpleAgent
# from llama_index.prompts import Prompt
# from llama_index.node_parser import SimpleNodeParser

# # Create an index from documents
# document_text = summary
# index = GPTSimpleVectorIndex.from_documents([document_text])

# # Define the SimpleAgent
# class TurningPointAgent(SimpleAgent):
#     def _run(self, query):
#         # Implement your turning point detection logic here
#         if "turning point" in query.lower():
#             return "The turning point is identified based on your criteria."
#         return "No turning point found."

# # Create the agent and run the analysis
# agent = TurningPointAgent(service_context=index)
# response = agent.run("Where is the turning point in this document?")
# print(response)

# Pass Book Summary thru the First Agent

In [41]:
def query_openai(system_prompt, user_prompt):
    """Send a prompt to OpenAI and return the response."""
    
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    return completion.choices[0].message.content

# response_format={“type”: “json_object”}

In [42]:
# Load the system prompt from system_prompts/story_analysis_prompt.md
with open("system_prompts/story_analysis_prompt.md", "r") as file:
    system_prompt = file.read()

# Generate a response using the OpenAI API
trailer_description = query_openai(system_prompt, summary)


# Create Specific clip/audio Prompts from the long description

In [56]:
import re

def time_to_seconds(timestr: str) -> int:
    """
    Convert a time string in MM:SS format to an integer number of seconds.
    E.g. "0:10" -> 10, "1:05" -> 65, etc.
    """
    minutes_str, seconds_str = timestr.split(':')
    return int(minutes_str) * 60 + int(seconds_str)

def parse_trailer_script(summary):
    """Send a prompt to OpenAI and return the response."""

    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "Remove the **s and output this as a json with keys for the clip number, visual, and audio information." + str(summary)}]
    # tools=tools
    )
    
    result = completion.choices[0].message.content
    cleaned_input = result.replace("```json\n", "").replace("```", "")
    return cleaned_input
    # Explanation of this regex:
    #
    # 1. \*\*Clip (\d+): (.*?)\*\*:
    #    - Matches "**Clip 1: Title**"
    #    - group(1) = "1", group(2) = "Establishing the World"
    #
    # 2. \s*-\s*\*\*Visual \((\d+:\d+)-(\d+:\d+)\)\:\*\*\s*(.*?)
    #    - Matches "- **Visual (0:00-0:10):**"
    #      group(3) = "0:00", group(4) = "0:10"
    #      group(5) = everything up until we hit the next "- **Audio:**" line
    #
    # 3. \s*-\s*\*\*Audio:\*\*\s*(.*?)\s*(?=\*\*Clip|\Z)
    #    - Matches "- **Audio:** " line, capturing everything up to the next "**Clip"
    #      or the end of text (\Z).
    #
    # We use DOTALL so that (.*?) can include newlines.
    #
    pattern = re.compile(
        r"\*\*Clip (\d+): (.*?)\*\*\s*"                # e.g. **Clip 1: Establishing the World**
        r"-\s*\*\*Visual \((\d+:\d+)-(\d+:\d+)\)\:\*\*\s*(.*?)\s*"  
        r"-\s*\*\*Audio:\*\*\s*(.*?)\s*(?=\*\*Clip|\Z)",
        re.DOTALL
    )

    clips = []
    matches = pattern.findall(script_text)
    for match in matches:
        clip_num_str, clip_title, start_str, end_str, visual_desc, audio_desc = match

        start_seconds = time_to_seconds(start_str)
        end_seconds = time_to_seconds(end_str)
        length_seconds = end_seconds - start_seconds

        clip_info = {
            "clip_number": int(clip_num_str),
            "clip_title": clip_title.strip(),
            "start_time": start_str,
            "end_time": end_str,
            "visual_description": visual_desc.strip(),
            "audio_description": audio_desc.strip(),
            "length_seconds": length_seconds,
        }
        clips.append(clip_info)

    return clips


In [57]:
cleaned_input = trailer_clips.replace("```json\n", "").replace("```", "")
print(cleaned_input)

{
  "Trailer Music Description": "The trailer will be underscored by a tense electronic score, featuring pulsating synths and percussive elements that build steadily in intensity. Layered with ambient soundscapes, the music will evoke a biopunk future, stirring a sense of urgency and suspense interwoven with moments of haunting beauty.",
  "Clip 1": {
    "Time": "0:00 - 0:15",
    "Visual": "A sweeping aerial shot over a flooded future Bangkok, where towering skyscrapers break through a shimmering ocean. Levees and massive pumps strain against the encroaching water.",
    "Audio": "The hum of machinery and distant city sounds play as a somber voiceover begins, 'In a world devoured by oceans and choked by engineered plagues...'"
  },
  "Clip 2": {
    "Time": "0:16 - 0:30",
    "Visual": "Quick cuts of genetically modified crops, luminous in the night, surrounded by shadowy figures in biohazard suits. A close-up on a sterile seed being exchanged for currency.",
    "Audio": "The rustli

In [51]:
print(trailer_description)

# import json
# json.loads(trailer_clips)

**Trailer Music Description:**
The trailer will be underscored by a tense electronic score, featuring pulsating synths and percussive elements that build steadily in intensity. Layered with ambient soundscapes, the music will evoke a biopunk future, stirring a sense of urgency and suspense interwoven with moments of haunting beauty.

---

**Clip 1: (0:00 - 0:15)**
- **Visual:** A sweeping aerial shot over a flooded future Bangkok, where towering skyscrapers break through a shimmering ocean. Levees and massive pumps strain against the encroaching water.
- **Audio:** The hum of machinery and distant city sounds play as a somber voiceover begins, "In a world devoured by oceans and choked by engineered plagues..."

**Clip 2: (0:16 - 0:30)**
- **Visual:** Quick cuts of genetically modified crops, luminous in the night, surrounded by shadowy figures in biohazard suits. A close-up on a sterile seed being exchanged for currency.
- **Audio:** The rustling of leaves and soft whispers of a transa